1. Configure Unity catalog
2. Create the project catalog
3. Create a Bronze Database in external location (Bz container)
4. Use Bz container to store the Bz tables   

1. Create an external location using Cat. Exp. UI to point to bronze layer  

In [0]:
%sql 
DESCRIBE EXTERNAL LOCATION `bz-layer`;

name,url,credential_name,owner,created_by,created_at,comment
bz-layer,abfss://bronze@datalakeselectivaproject.dfs.core.windows.net/,951e8605-602e-416c-bc17-7247f045b98e-storage-credential-1737568724489,agampaul.s@outlook.com,agampaul.s@outlook.com,2025-01-22T18:39:54.995Z,null


Create the directory structure to STORE the bronze layer tables data  

In [0]:
# Create the directory structure inside bz-layer using the directories of the landing zone 
bz_layer_path = 'abfss://bronze@datalakeselectivaproject.dfs.core.windows.net/'
landing_zone_path = '/mnt/landing-zone/'
expected_bz_tables = ['channels/', 'costs/', 'customers/', 'products/', 'promotions/', 'sales/', 'supplementary_demographics/', 'times/']
table_list = dbutils.fs.ls(bz_layer_path)
tables_names = [i.name for i in table_list]
if len(table_list) != 8 or expected_bz_tables != tables_names: 
    for itr in dbutils.fs.ls(landing_zone_path):
        dir_name = itr.name 
        dbutils.fs.mkdirs(bz_layer_path + dir_name)
        print(dir_name)

print(tables_names)

['channels/', 'costs/', 'customers/', 'products/', 'promotions/', 'sales/', 'supplementary_demographics/', 'times/']


In [0]:
%sql
CREATE CATALOG IF NOT EXISTS selectiva-project;

Create the SCHEMA/DB inside the project catalog 
* Only stores the metadata for the tables, the actual table data and schema strored inside the 'bz-layer' external location

In [0]:
%sql 
CREATE SCHEMA IF NOT EXISTS `selectiva-project`.bz